In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# 1. Colab 환경 설정

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3.0.2
!pip install torch

In [ ]:
#깃허브에서 SKTBrain KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [ ]:
# 한국어 띄어쓰기 패키지
!pip install git+https://github.com/haven-jeon/PyKoSpacing.git
from pykospacing import Spacing

In [ ]:
# 네이버 한글 맞춤법 검사기 기반 패키지
!pip install git+https://github.com/ssut/py-hanspell.git
from hanspell import spell_checker

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-932ip3pm
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-932ip3pm


In [ ]:
# 한국어 문장 분리기
!pip install kss
import kss

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
#from tqdm import tqdm, tqdm_notebook
from tqdm import notebook

#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#!nvidia-smi

#2. 데이터 로드 및 전처리

In [ ]:
import pandas as pd
df = pd.read_excel('/content/drive/MyDrive/Tave Research/삼성카드 공모전/공모전_제공_데이터(1차).xlsx', sheet_name='학습데이터')
rank_df = pd.read_excel('/content/drive/MyDrive/Tave Research/삼성카드 공모전/공모전_제공_데이터(1차).xlsx', sheet_name='우선순위')
display(df.head(), rank_df.head())

,NO,NO2,발화,발화1,발화1의 우선순위,발화2,발화2의 우선순위,발화3,발화3의 우선순위,최종분류(우선순위 가장 높은것 선택)
0,1,6438,상담직원과 빠른연결했슴좋겠어요,불만>고객서비스>상담시스템,5,NaN,NaN,NaN,NaN,불만>고객서비스>상담시스템
1,2,6850,안기다리고. 빨리상담되었음좋겠다,불만>고객서비스>상담시스템,5,NaN,NaN,NaN,NaN,불만>고객서비스>상담시스템
2,3,7294,너무 오래기다리고 상담이 늦어져요,불만>고객서비스>상담시스템,5,NaN,NaN,NaN,NaN,불만>고객서비스>상담시스템
3,4,9484,상담직원과 통화가 매우 힘들다,불만>고객서비스>상담시스템,5,NaN,NaN,NaN,NaN,불만>고객서비스>상담시스템
4,5,10350,상담받으려면오래기다리는게불편함,불만>고객서비스>상담시스템,5,NaN,NaN,NaN,NaN,불만>고객서비스>상담시스템


,우선순위,의도명(유형)
0,1,칭찬>고객서비스>상담원
1,2,칭찬>고객서비스>상담시스템
2,3,칭찬>고객서비스
3,4,불만>고객서비스>상담원
4,5,불만>고객서비스>상담시스템


In [ ]:
df_origin = df.copy()

In [ ]:
# 네이버 영화 리뷰데이터
!git clone https://github.com/e9t/nsmc.git 
train = pd.read_csv("nsmc/ratings_train.txt", sep='\t')
test = pd.read_csv("nsmc/ratings_test.txt", sep='\t')

fatal: destination path 'nsmc' already exists and is not an empty directory.


In [ ]:
train.shape, test.shape

((150000, 3), (50000, 3))

In [ ]:
train

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
...,...,...,...
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1


In [ ]:
#rank_df['우선순위'].unique()

In [ ]:
# 중립 데이터 7건 제거
# df = df.drop(df[df['최종분류(우선순위 가장 높은것 선택)'].str.contains('중립')].index)
# df.reset_index(inplace=True, drop=True)

In [ ]:
# 최종분류 컬럼에서 불만:0, 칭찬:1, (중립:1)
df['class'] = df['최종분류(우선순위 가장 높은것 선택)'].apply(lambda x: 0 if '불만' in x else 1)
df = df[['발화', 'class']]
df = df.rename(columns={'발화':'sentence'})
df['class'].value_counts()

0    516
1    477
Name: class, dtype: int64

In [ ]:
df.head()

,발화,발화1의 우선순위,발화2의 우선순위,발화3의 우선순위,class
0,상담직원과 빠른연결했슴좋겠어요,5,NaN,NaN,0
1,안기다리고. 빨리상담되었음좋겠다,5,NaN,NaN,0
2,너무 오래기다리고 상담이 늦어져요,5,NaN,NaN,0
3,상담직원과 통화가 매우 힘들다,5,NaN,NaN,0
4,상담받으려면오래기다리는게불편함,5,NaN,NaN,0


In [ ]:
## 32개
#df_origin[df_origin['발화2의 우선순위'].notnull()]

In [ ]:
import re

def text_cleaning(docs):
    
    # 특수기호, 숫자, 자음, 모음 제거
    pattern1 = re.compile("[^ a-zA-Z가-힣]")
    docs = [pattern1.sub("", doc) for doc in docs]
    
    # 이중 띄어쓰기 제거
    docs = [re.sub(' +', ' ', doc) for doc in docs]

    # 띄어쓰기 검사 -> py-hanspell에서 띄어쓰기 없이 연속된 문장에 대한 오류 개선이 안 되어있어 띄어쓰기 검사를 제공하는 pykospacing 함께 사용
    spacing = Spacing()
    docs = [spacing(doc) for doc in docs]

    # 맞춤법 검사
    docs = [spell_checker.check(doc).checked for doc in docs]

    return docs

In [ ]:
df['cleaned_sentence'] = text_cleaning(df['발화'])

In [ ]:
df['rank_list'] = 'NaN'

for i in range(len(df)):
  df['rank_list'][i] = list([df['발화1의 우선순위'][i], df['발화2의 우선순위'][i], df['발화3의 우선순위'][i]])

df.head()

,발화,발화1의 우선순위,발화2의 우선순위,발화3의 우선순위,class,cleaned_sentence,rank_list
0,상담직원과 빠른연결했슴좋겠어요,5,NaN,NaN,0,상담직원과 빠른 연결했음 좋겠어요,"[5, nan, nan]"
1,안기다리고. 빨리상담되었음좋겠다,5,NaN,NaN,0,안 기다리고 빨리 상담되었으면 좋겠다,"[5, nan, nan]"
2,너무 오래기다리고 상담이 늦어져요,5,NaN,NaN,0,너무 오래 기다리고 상담이 늦어져요,"[5, nan, nan]"
3,상담직원과 통화가 매우 힘들다,5,NaN,NaN,0,상담직원과 통화가 매우 힘들다,"[5, nan, nan]"
4,상담받으려면오래기다리는게불편함,5,NaN,NaN,0,상담받으려면 오래 기다리는 게 불편함,"[5, nan, nan]"


In [ ]:
# KSS로 복문 분리 확인
import kss

df_final = pd.DataFrame(data='NaN', columns=['splited_sentence', 'multi_class'], index=range(len(df)*2))

cnt = 0

for i in range(len(df)):
  sentences = kss.split_sentences(df.iloc[i]['cleaned_sentence'])
  
  for j in range(len(sentences)):
    df_final.iloc[cnt]['splited_sentence'] = sentences[j]

    if len(sentences) == 1:
      df_final.iloc[cnt]['multi_class'] = min(df.iloc[i]['rank_list'])
      cnt = cnt + 1
      continue
    
    if j >= len(df.iloc[i]['rank_list']): # rank_list의 총 개수(3개)보다 더 많이 분리된 문장일 경우 rank_list의 min 값으로 넣어줌
      df_final.iloc[cnt]['multi_class'] = min(df.iloc[i]['rank_list'])
      cnt = cnt + 1
      continue

    if str(df.iloc[i]['rank_list'][j]) == 'nan': # 해당 rank_list 자리가 'nan'이라면 rank_list의 min 값으로 넣어줌
      df_final.iloc[cnt]['multi_class'] = min(df.iloc[i]['rank_list'])
      cnt = cnt + 1

    else:
      df_final.iloc[cnt]['multi_class'] = df.iloc[i]['rank_list'][j]
      cnt = cnt + 1


In [ ]:
display(df_final.head(), df_final.tail())

,splited_sentence,multi_class
0,상담직원과 빠른 연결했음,5
1,좋겠어요,5
2,안 기다리고 빨리 상담되었으면 좋겠다,5
3,너무 오래 기다리고 상담이 늦어져요,5
4,상담직원과 통화가 매우 힘들다,5


,splited_sentence,multi_class
1981,NaN,NaN
1982,NaN,NaN
1983,NaN,NaN
1984,NaN,NaN
1985,NaN,NaN


In [ ]:
df_final = df_final[df_final['splited_sentence']!='NaN']
df_final.tail()

,splited_sentence,multi_class
1149,잃어버린 지도 몰랐는데 감사합니다,11
1150,수시로 개월 무이자 할부와 링크 혜택이 다른 카드와 구별되는 기능이 편리함,7
1151,무이자 할부 자주 많이 주셔서 감사합니다,7
1152,무이자 혜택 만족 긴 할부 만족 잘 쓰고 있습니다,7
1153,무이자 혜택이 많아 잘 쓰고 있어요,7


In [ ]:
#[print(i, df['kss_tokenized'].loc[i]) for i in range(len(df)) if len(df['kss_tokenized'].loc[i]) > 1]

# 3.Train data & Test data

In [ ]:
df_final['multi_class'] = df_final['multi_class'].astype(int)
print(df_final['multi_class'].unique())
len(df_final['multi_class'].unique())

[ 5 33 24 23  4 42 41 37 35 34 32 31 30 29 26 25  2 14  1 19 18 15 13 12
 11  7]


26

In [ ]:
df_final['multi_class'].value_counts()

1     321
4     191
33    164
24    147
5     133
2      42
30     38
25     23
23     21
37     17
42      8
15      8
32      7
7       4
41      4
26      4
34      4
18      3
13      3
35      2
29      2
11      2
19      2
14      2
12      1
31      1
Name: multi_class, dtype: int64

In [ ]:
def convert_class(class_):
  sorted_classes = np.sort(df_final['multi_class'].unique())
  for i in range(len(sorted_classes)):
    if class_ == sorted_classes[i]:
      class_ = np.where(sorted_classes == sorted_classes[i])[0][0]
  return class_

In [ ]:
import numpy as np

sorted_classes = np.sort(df_final['multi_class'].unique())

print(sorted_classes)
print(np.where(sorted_classes == 1))

[ 1  2  4  5  7 11 12 13 14 15 18 19 23 24 25 26 29 30 31 32 33 34 35 37
 41 42]
(array([0]),)


In [ ]:
df_final['multi_class'] = df_final['multi_class'].apply(convert_class)

In [ ]:
len(df_final['multi_class'].unique())

26

In [ ]:
data_list = []
for sen, label in zip(df_final['splited_sentence'], df_final['multi_class']):
  data = []
  data.append(sen)
  data.append(str(label))

  data_list.append(data)

data_list[0]

['상담직원과 빠른 연결했음', '3']

In [ ]:
# train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=0, shuffle=True)

print(len(dataset_train), len(dataset_test))

923 231


#4. KoBERT 입력 데이터 만들기

모든 학습 데이터는 배치단위로 입/출력이 정의되기 때문에 입력 데이터 처리와 레이블 처리를 동시에 배치 출력하기 위해 Dataset 클래스를 정의하면 아래와 같다. Dataset 클래스는 PyTorch의 Dataset과 동일한 형태를 띈다.

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

# BERTSentenceTransform 모듈을 통해 토큰화와 패딩이 이루어짐

In [ ]:
# Setting parameters
max_len = 64 # origin: 128
batch_size = 32 # origin: 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model


In [ ]:
dataset_train[0]

['공부 많이 하시고 상담받으십시오', '2']

In [ ]:
# 토큰화와 패딩 확인
data_train[0]

# 출력값은 3개의 array
# 첫 번째는 패딩된 시퀀스, 두 번째는 길이와 타입에 대한 내용, 세 번재는 어텐션 마스크 시퀀스
# BERT에 데이터가 입력되었을 때 어텐션 함수가 적용되어 연산이 된다. 이때 1로 패딩된 값들은 연산할 필요가 없기 때문에 연산을 하지 않아도 된다고 알려주는 데이터가 있어야 하는데 그게 바로 어텐션 마스크 시퀀스

(array([   2, 1042, 1956, 4924, 6705, 5439, 2659, 6288, 7074, 6749, 6705,
        6964,    3,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(13, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 2)

In [ ]:
# torch 형식의 dataset
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

# 5. KoBERT 학습 모델 만들기

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 26,   ##클래스 수 조정##
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
train_dataloader

#6. KoBERT 모델 학습시키기

In [ ]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
# 학습 데이터셋과 학습 모델 준비가 다 끝났다면 모델 학습시키기
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    torch.cuda.empty_cache()    
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    torch.cuda.empty_cache()

epoch 1 batch id 1 loss 3.3753764629364014 train acc 0.0

epoch 1 train acc 0.15952266922094507



epoch 1 test acc 0.3364955357142857


epoch 2 batch id 1 loss 2.658581495285034 train acc 0.34375

epoch 2 train acc 0.46008939974457214



epoch 2 test acc 0.5005580357142857


epoch 3 batch id 1 loss 1.8381327390670776 train acc 0.5625

epoch 3 train acc 0.6350574712643678


...


epoch 8 test acc 0.8297991071428571


epoch 9 batch id 1 loss 0.6795845627784729 train acc 0.84375

epoch 9 train acc 0.9058508939974458



epoch 9 test acc 0.8219866071428571


epoch 10 batch id 1 loss 0.687133252620697 train acc 0.875

epoch 10 train acc 0.9220146871008941



epoch 10 test acc 0.8297991071428571


 # 7. 새로운 문장 테스트

In [ ]:
def pred_text_cleaning(text):
    
    # 특수기호, 숫자, 자음, 모음 제거
    pattern1 = re.compile("[^ a-zA-Z가-힣]")
    text = pattern1.sub("", text)
    
    # 이중 띄어쓰기 제거
    text = re.sub(' +', ' ', text)

    # 띄어쓰기 검사 -> py-hanspell에서 띄어쓰기 없이 연속된 문장에 대한 오류 개선이 안 되어있어 띄어쓰기 검사를 제공하는 pykospacing 함께 사용
    spacing = Spacing()
    text = spacing(text)

    # 맞춤법 검사
    text = spell_checker.check(text).checked

    return text

In [ ]:
# 입력되는 문장을 KoBERT의 입력 형식으로 바꿔주는 코드를 작성
# 토큰화, 패딩, 텐서를 바꿔주고 예측을 하는 'predict' 함수를 만들기
# 토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

convert_origin_class = lambda x: sorted_classes[x]

def predict(predict_sentence):
    pred_result = []
    predict_sentence = pred_text_cleaning(predict_sentence)
    predict_sentences = kss.split_sentences(predict_sentence)

    for j in range(len(predict_sentences)):
    
      data = [predict_sentences[j], '0']
      dataset_another = [data]

      another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
      test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
      
      model.eval()

      for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
          token_ids = token_ids.long().to(device)
          segment_ids = segment_ids.long().to(device)

          valid_length= valid_length
          label = label.long().to(device)

          out = model(token_ids, valid_length, segment_ids)

          test_eval=[]
          for i in out:
              logits = i
              logits = logits.detach().cpu().numpy()

                # if np.argmax(logits) == 0:
                #     test_eval.append("불만")
                # elif np.argmax(logits) == 1:
                #     test_eval.append("만족")
              test_eval.append(np.argmax(logits))
              print()

          #print(">> 해당 발화는 " + test_eval[0] + " 피드백입니다.")
          pred_result.append(test_eval[0])

    pred_result = list(map(convert_origin_class, pred_result))
    print(predict_sentences, pred_result)
    result = min(pred_result)

    print('발화 유형:', rank_df.loc[result-1]['의도명(유형)'])

    if '불만' in rank_df.loc[result-1]['의도명(유형)']:
      print('해당 발화는 불만 피드백입니다.')
    else:
      print('해당 발화는 만족 피드백입니다.')


    return pred_result

using cached model


In [ ]:
#질문 무한반복하기! 0 입력시 종료
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == '0' :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 마스크를 쓰고잇는건 알지만 발음이 너무 부정확해서 뭐라하는지 모르겠어요

['마스크를 쓰고 있는 건 알지만 발음이 너무 부 정확해서 뭐라 하는지 모르겠어요'] [4]
발화 유형: 불만>고객서비스>상담원
해당 발화는 불만 피드백입니다.


하고싶은 말을 입력해주세요 : 0
